In [2]:
import tensorflow as tf
import os
import pickle
import numpy as np
import slim
CIFAR_DIR = "../cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))
import matplotlib.pyplot as plt

['.ipynb_checkpoints', '0.py', '2.csv', '3.csv', '3chTest', '3chTest1', '3chTrain', '3chTrain1', '3chTrain2', '4.csv', '5.csv', 'a.jpg', 'batches.meta', 'cDCGAN_VS_CNN-Copy1.ipynb', 'cDCGAN_VS_CNN.ipynb', 'checkpoint', 'data', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'imageChangeChannel.ipynb', 'readme.html', 'resize.py', 'splite_data.py', 'SSGAN_VS_CNN.ipynb', 'test', 'test.png', 'test1.png', 'test2', 'test2.png', 'test3', 'testNew', 'test_batch', 'train', 'train2', 'train3', 'trainNew', 'Untitled-Copy1.ipynb', 'Untitled1.ipynb']


In [3]:
import os 
import numpy as np
from PIL import Image
def load_Traindata():
    train = []
    file_names = os.listdir('./train/')
    for i in range(len(file_names)):
        im = Image.open('./train/'+file_names[i])
        im2 = np.array(im)
        train.append(im2)
    train2 = np.array(train)
    
    #test2 = np.array(test)
    train_lable = np.zeros((len(train2),1),dtype='int8')
    #test_lable = np.zeros((len(test2),1),dtype='int8')
    for i in range(6):
        for j in range(i*240, (i+1)*240):
            train_lable[j,0] = i
   
    train_lable = train_lable.reshape((train_lable.shape[0],))
    return (train2, train_lable)
def load_Testdata():
    test = []
    file_names = os.listdir('./test/')
    for i in range(len(file_names)):
        im = Image.open('./test/'+file_names[i])
        im2 = np.array(im)
        test.append(im2)
    test2 = np.array(test)
    #train_lable = np.zeros((len(train2),1),dtype='int8')
    test_lable = np.zeros((len(test2),1),dtype='int8')
    for i in range(6):
        for j in range(i*60, (i+1)*60):
            test_lable[j,0] = i
    test_lable = test_lable.reshape((test_lable.shape[0],))
    return (test2, test_lable)

In [6]:
class SteelData:
    def __init__(self,  need_shuffle, isTrain):
        all_data = []
        all_labels = []
        if isTrain:
            data, labels = load_Traindata()
        else:
            data, labels = load_Testdata()
        all_data.append(data)
        all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        #print(self._data.shape)
        #print(self._labels.shape)
        #print(self._labels)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        #print(batch_labels)
        self._indicator = end_indicator
        return batch_data, batch_labels

#train_filenames = [os.path.join(CIFAR_DIR, 'train' % i) for i in range(1, 6)]
#test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = SteelData(True, True)
test_data = SteelData(False, False)

In [18]:
x = tf.placeholder(tf.float32, [None, 32, 32, 3])
#x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
#x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
#x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# conv1: 神经元图， feature_map, 输出图像
conv1 = tf.layers.conv2d(x,
                         64, # output channel number
                         (5,5), # kernel size
                         strides=(2, 2),
                         padding = 'same',
                         activation = tf.nn.relu,
                         )
bn1 = tf.contrib.layers.batch_norm(conv1, center=True, scale=True,
                                          decay=0.9,
                                          updates_collections=None)
d1 = tf.layers.dropout(
    bn1,
    rate=0.5,
    noise_shape=None,
    seed=None,
    training=False,
    name=None
    )


conv2 = tf.layers.conv2d(d1,
                         128, # output channel number
                         (5,5), # kernel size
                         strides=(2, 2),
                         padding = 'same',
                         activation = tf.nn.relu,
                         )
bn2 = tf.contrib.layers.batch_norm(conv2, center=True, scale=True,
                                          decay=0.9, 
                                          updates_collections=None)
d2 = tf.layers.dropout(
    bn2,
    rate=0.5,
    noise_shape=None,
    seed=None,
    training=False,
    name=None
    )

conv3 = tf.layers.conv2d(d2,
                         256, # output channel number
                         (5,5), # kernel size
                         strides=(2, 2),
                         padding = 'same',
                         activation = tf.nn.relu,
                         )
bn3 = tf.contrib.layers.batch_norm(conv3, center=True, scale=True,
                                          decay=0.9, 
                                          updates_collections=None)
d3 = tf.layers.dropout(
    bn3,
    rate=0.5,
    noise_shape=None,
    seed=None,
    training=False,
    name=None
    )

# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(d3)
y_ = tf.layers.dense(flatten, 6)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
init = tf.global_variables_initializer()
batch_size = 60
train_steps = 10000
test_steps = 6

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = SteelData(True,False)
            all_test_acc_val = []
            all_pred = []
            all_pred = np.array(all_pred)
            all_labels = []
            all_labels = np.array(all_labels)
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
#                 test_acc_val = sess.run(
#                     [accuracy],
#                     feed_dict = {
#                         x: test_batch_data, 
#                         y: test_batch_labels
#                     })
                test_acc_val, preds = sess.run(
                        [accuracy, predict],
                        feed_dict = {
                            x: test_batch_data, 
                            y: test_batch_labels
                        })
                print(preds.shape)
                print(test_batch_labels.shape)
                print(preds)
                print(test_batch_labels)
                all_pred = np.hstack((all_pred, preds))
                all_labels = np.hstack((all_labels, test_batch_labels))
#                 all_pred.append(preds)
#                 all_labels.append(test_batch_labels)
            #labels = tf.arg_max(labels, dimension=1)
            #preds = tf.arg_max(preds, dimension=1)
#                 confusion_matrix = tf.confusion_matrix(predictions=preds, labels=test_batch_labels, num_classes=6, dtype=tf.int32, name=None, weights=None)
#                 confusion_matrix1 = sess.run(confusion_matrix)
#                 print(confusion_matrix1)
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            confusion_matrix = tf.confusion_matrix(predictions=tf.convert_to_tensor(all_pred), 
                                                   labels=tf.convert_to_tensor(all_labels), 
                                                   num_classes=6, dtype=tf.int32, 
                                                   name=None, weights=None)
            confusion_matrix1 = sess.run(confusion_matrix)
            print(confusion_matrix1)
            print('[Test ] Step: %d, acc: %4.5f'
                  % (i+1, test_acc))

# train 10k: 71.35%
# with tf.Session() as sess:
#     sess.run(init)
#     all_steps = []
#     all_train_acc = []
#     all_test_steps = []
#     all_test_acc = []
#     all_test_acc_val = []
#     all_loss = []
#     for i in range(train_steps):
        
        
        
#         batch_data, batch_labels = train_data.next_batch(batch_size)
#         #print(batch_labels)
#         loss_val, acc_val, _ = sess.run(
#             [loss, accuracy, train_op],
#             feed_dict={
#                 x: batch_data,
#                 y: batch_labels})
#         if (i+1) % 100 == 0:
#             print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
#                   % (i+1, loss_val, acc_val))
#             all_steps.append(i)
#             all_train_acc.append(acc_val)
#             all_loss.append(loss_val)
#             #if (i+1) % 1000 == 0:
#             #test_data = SteelData(False,False)


#             test_batch_data, test_batch_labels \
#                         = test_data.next_batch(batch_size)
#             #print(test_batch_labels)
#             test_acc_val, preds = sess.run(
#                         [accuracy, predict],
#                         feed_dict = {
#                             x: test_batch_data, 
#                             y: test_batch_labels
#                         })
#             print(preds.shape)
#             print(test_batch_labels.shape)
#             print(preds)
#             print(test_batch_labels)
#             #labels = tf.arg_max(labels, dimension=1)
#             #preds = tf.arg_max(preds, dimension=1)
#             confusion_matrix = tf.confusion_matrix(predictions=preds, labels=test_batch_labels, num_classes=6, dtype=tf.int32, name=None, weights=None)
#             confusion_matrix1 = sess.run(confusion_matrix)
#             print(confusion_matrix1)
#             all_test_acc_val.append(test_acc_val)

#             all_test_steps.append(i)
#             test_acc = np.mean(all_test_acc_val)
        
#         #all_test_acc.append(test_acc)
#             print('[Test ] Step: %d, acc: %4.5f'
#                   % (i+1, test_acc))
#     #print(all_steps)      
#     #print(all_train_acc)
#     fig,ax = plt.subplots()
 
#     #plt.xlabel('migration speed (MB/s)')
#     #plt.ylabel('migration time (s); request delay (ms)')

#     """set interval for y label"""
#     #yticks = range(0,1,11)
#     #ax.set_yticks(yticks)


#     """set min and max value for axes"""
#     #ax.set_ylim([0,1])
#     ax.set_xlim([0,10000])


#     #x = [57,56,55,54,53,52,51,50,49,48,47,46,45,44,43]
#     line1, = plt.plot(all_steps,all_train_acc,label="train accrucy")
#     line2, =plt.plot(all_steps,all_test_acc_val,'-.',label="test accrucy")
#     line3, =plt.plot(all_steps,all_loss,'--',label="test accrucy")
#     #plt.plot(all_steps,all_test_acc_val,"+-",label="request delay")

#     """open the grid"""
#     plt.grid(True)

#     #plt.legend(bbox_to_anchor=(1.0, 1), loc=1, borderaxespad=0.)
#     plt.legend([line1, line2,line3], ['train accrucy', 'test accrucy','loss'], loc = 'best') 
#     #plt.savefig('./test.png')
#     plt.savefig('./test1.png')
#     plt.show()




[Train] Step: 100, loss: 0.32172, acc: 0.86667
[Train] Step: 200, loss: 0.18884, acc: 0.93333
[Train] Step: 300, loss: 0.10019, acc: 0.95000
[Train] Step: 400, loss: 0.11459, acc: 0.95000
[Train] Step: 500, loss: 0.07497, acc: 0.98333
[Train] Step: 600, loss: 0.17683, acc: 0.95000
[Train] Step: 700, loss: 0.11833, acc: 0.96667
[Train] Step: 800, loss: 0.03629, acc: 0.98333
[Train] Step: 900, loss: 0.00483, acc: 1.00000
[Train] Step: 1000, loss: 0.01542, acc: 1.00000
(60,)
(60,)
[1 4 0 5 5 1 4 4 2 0 0 0 2 3 4 3 3 0 4 0 0 5 3 1 0 0 0 3 0 1 4 3 3 2 3 5 3
 2 1 0 3 3 0 0 0 1 1 1 4 4 0 1 3 1 5 4 3 4 4 5]
[1 4 4 5 5 3 4 4 2 0 0 0 2 3 0 3 3 0 4 5 4 2 3 5 0 0 0 5 0 1 4 3 3 2 3 5 3
 2 5 0 3 0 0 5 0 5 1 1 4 4 4 1 5 1 5 4 3 4 4 5]
(60,)
(60,)
[1 1 2 4 0 3 0 1 1 1 2 0 4 4 5 3 2 1 4 5 0 2 0 3 1 4 2 2 2 4 3 0 4 2 2 1 1
 2 1 2 2 5 3 3 1 3 1 4 4 0 5 2 5 1 1 2 2 1 0 0]
[1 1 2 1 0 3 0 1 1 1 2 0 4 4 5 5 2 5 4 5 3 2 0 5 1 4 2 2 2 4 3 4 5 2 2 1 3
 2 1 2 2 5 3 3 1 3 3 4 4 0 5 2 5 1 1 2 2 1 0 0]
(60,)
(60,)
[

[[52  0  5  1  2  0]
 [ 0 54  0  1  2  3]
 [ 0  0 57  1  0  2]
 [ 3  5  0 52  0  0]
 [ 8  0  0  0 52  0]
 [ 1  6  0  8  3 42]]
[Test ] Step: 4000, acc: 0.85833
[Train] Step: 4100, loss: 0.00004, acc: 1.00000
[Train] Step: 4200, loss: 0.00002, acc: 1.00000
[Train] Step: 4300, loss: 0.00002, acc: 1.00000
[Train] Step: 4400, loss: 0.00002, acc: 1.00000
[Train] Step: 4500, loss: 0.00002, acc: 1.00000
[Train] Step: 4600, loss: 0.00002, acc: 1.00000
[Train] Step: 4700, loss: 0.00004, acc: 1.00000
[Train] Step: 4800, loss: 0.00003, acc: 1.00000
[Train] Step: 4900, loss: 0.00001, acc: 1.00000
[Train] Step: 5000, loss: 0.00001, acc: 1.00000
(60,)
(60,)
[2 0 2 0 5 3 2 2 4 0 1 2 5 2 3 3 1 4 5 5 3 3 2 4 4 2 5 1 1 0 0 4 1 4 1 3 0
 0 2 2 0 3 1 1 5 3 3 3 1 4 4 4 1 1 5 3 1 4 0 2]
[2 0 2 0 5 3 2 2 4 3 1 2 5 2 3 3 1 4 2 5 3 5 2 4 4 2 5 1 1 0 0 4 1 4 1 3 4
 0 2 2 0 3 3 1 1 1 3 3 1 5 4 5 1 3 5 5 1 1 0 2]
(60,)
(60,)
[0 5 5 5 2 4 2 4 2 3 4 1 3 4 0 3 2 2 4 5 0 1 0 5 2 3 5 4 2 3 5 5 3 1 4 1 0
 4 0 2 2 3 0 4 

[[54  0  3  2  1  0]
 [ 0 49  0  2  4  5]
 [ 2  0 57  0  0  1]
 [ 2  6  1 50  0  1]
 [10  0  0  0 49  1]
 [ 1  5  0  7  3 44]]
[Test ] Step: 8000, acc: 0.84167
[Train] Step: 8100, loss: 0.00015, acc: 1.00000
[Train] Step: 8200, loss: 0.00023, acc: 1.00000
[Train] Step: 8300, loss: 0.00004, acc: 1.00000
[Train] Step: 8400, loss: 0.00003, acc: 1.00000
[Train] Step: 8500, loss: 0.01150, acc: 1.00000
[Train] Step: 8600, loss: 0.13894, acc: 0.96667
[Train] Step: 8700, loss: 0.05558, acc: 0.98333
[Train] Step: 8800, loss: 0.00306, acc: 1.00000
[Train] Step: 8900, loss: 0.00279, acc: 1.00000
[Train] Step: 9000, loss: 0.00393, acc: 1.00000
(60,)
(60,)
[0 4 1 1 3 1 2 5 0 4 0 1 1 3 2 0 0 4 5 4 1 0 2 4 4 5 4 0 4 4 3 4 2 1 2 2 0
 5 5 4 1 1 2 3 0 2 5 1 1 5 3 5 3 2 3 3 0 2 0 0]
[0 4 1 1 3 1 2 5 0 4 3 1 5 3 2 5 0 5 1 1 1 0 2 1 4 5 5 0 4 4 3 4 2 1 2 2 0
 5 5 4 1 1 2 3 0 2 5 1 1 5 3 5 3 2 3 3 0 2 0 0]
(60,)
(60,)
[3 5 3 5 3 1 1 4 2 5 3 1 0 4 5 4 0 4 2 3 2 4 1 1 5 3 0 4 4 2 1 2 5 4 0 4 2
 2 5 5 2 4 4 2 

In [ ]:
print(all_test_acc_val)

In [ ]:
fig,ax = plt.subplots()
 
#plt.xlabel('migration speed (MB/s)')
    #plt.ylabel('migration time (s); request delay (ms)')

    #"""set interval for y label"""
    #yticks = range(0,1,11)
    #ax.set_yticks(yticks)


    #"""set min and max value for axes"""
ax.set_ylim([0,1.1])
ax.set_xlim([0,10000])


    #x = [57,56,55,54,53,52,51,50,49,48,47,46,45,44,43]
line1, = plt.plot(all_steps,all_train_acc,label="train accrucy")
line2, =plt.plot(all_steps,all_test_acc_val,'-.',label="test accrucy")
    #plt.plot(all_steps,all_test_acc_val,"+-",label="request delay")

 #"""open the grid"""
plt.grid(True)

    #plt.legend(bbox_to_anchor=(1.0, 1), loc=1, borderaxespad=0.)
plt.legend([line1, line2], ['train accrucy', 'test accrucy'], loc = 'best') 
plt.savefig('./test.png')
plt.show()

In [ ]:
fig,ax = plt.subplots()
 
#plt.xlabel('migration speed (MB/s)')
    #plt.ylabel('migration time (s); request delay (ms)')

    #"""set interval for y label"""
    #yticks = range(0,1,11)
    #ax.set_yticks(yticks)


    #"""set min and max value for axes"""
#ax.set_ylim([0,1.1])
ax.set_xlim([0,10000])


    #x = [57,56,55,54,53,52,51,50,49,48,47,46,45,44,43]
#line1, = plt.plot(all_steps,all_train_acc,label="train accrucy")
#line2, =plt.plot(all_steps,all_test_acc_val,'-.',label="test accrucy")
    #plt.plot(all_steps,all_test_acc_val,"+-",label="request delay")

 #"""open the grid"""
plt.grid(True)
plt.plot(all_steps,all_loss,'--',label="test accrucy")
    #plt.legend(bbox_to_anchor=(1.0, 1), loc=1, borderaxespad=0.)
#plt.legend([line1, line2], ['train accrucy', 'test accrucy'], loc = 'best') 
plt.savefig('./test2.png')
plt.show()

In [ ]:
import csv

In [ ]:
row1 = all_train_acc
print(row1)

In [ ]:
row2 = all_steps

In [ ]:
data = list(map(lambda x:[x],row1))
with open('3.csv','w') as f:
    f_csv = csv.writer(f)
    for i in data:
            f_csv.writerow(i)
    #f_csv.writerows(row2)
    #f_csv.writerows(row1)

In [ ]:
np.mean(all_test_acc_val[-6:])